# Dogs vs Cats classification using transfer learning and CNN

# Import Required Libraries


This kernel shows how to use transfer learning to classify dogs and cats images. If you find this kernel useful please upvote it!

Transfer learning means that instead of your model learning everything from scratch, it uses another model that was trained on a similar problem, so that you can "transfer" the learned "knowledge" of the pretrained model to your model, and then learn some new features.

The ImageNet Data set is huge data set consisting of more that 14 million images from more than 22,000 different categories, here we are using a smaller version of it which has 1000 different categories.

In this kernel we use an xception model which is pretrained on the ImageNet dataset and then build some layers on top of it to be able to classify dogs and cats images.

Transfer learning makes sense here because the ImageNet data set has a much larger number of images (14 million) than the dog-cat data set (25,000). This increases the speed of training of our model and the accuracy of our predictions.

In [ ]:
import numpy as np
import pandas as pd
import random
import os
import matplotlib.pyplot as plt
import seaborn as sns
import keras.applications.xception as xception
import zipfile
import sys
import time
import tensorflow.keras as keras

from PIL import Image
from keras.layers import Input, Conv2D, Dense, Flatten, MaxPooling2D, Input
from keras.models import Model, Sequential
from keras.preprocessing import image
from keras.utils import to_categorical

print('setup successful!')
print(f'keras version {keras.__version__}')

# Define Constants

In [ ]:
IMAGE_WIDTH = 224
IMAGE_HEIGHT = 224
IMAGE_CHANNELS = 3

# The image net have 1000 categories, and so 1000 outputs, so we get 1000 different features
IMAGE_FEATURES_SIZE = 1000

# For the first run PERFORM_UNZIPPING must be True, but after the first run,
# it can be set to False skip the unzipping step to save time
PERFORM_UNZIPPING = True

print('defining constants successful!')

# Unzip the Train and Test folders

The train and test folders are zipped, the code below unzips both folders and the unzipped folders are saved in the output folder. For the First run PERFORM_UNZIPPING should be true to perform the unzipping, later if the folders are already unzipped you can change it to False

In [ ]:
if PERFORM_UNZIPPING == False:
    # Unzip the input folders
    with zipfile.ZipFile("../input/lion-data/trainlion/train", "r") as z:
        z.extractall(".")

    with zipfile.ZipFile("../input/lion-data/testlion/test", "r") as z:
        z.extractall(".")

    print('Unzipping done!')

else:
    print('Unzipping not needed')

imgs_path = "/kaggle/input/training-data-with-redondo/trainlion_redondo/trainlion_redondo/"

# Prepare load_image function and Data Frame
The pretrained model that we are using here is the xception model. The advantage of the xception model is that it has a small size, relatively small number of parameters and high accuracy. You can have a look with on a comparison of different pretrained models available in Keras in https://keras.io/api/applications/


The function 'load_image' takes the path of an image as an input and return the preprocessed image. 


In [ ]:
def _load_image(img_path):
    # load the image from the directory
    img = image.load_img(img_path, target_size=(IMAGE_WIDTH, IMAGE_HEIGHT))
    img = image.img_to_array(img)
    # add an additional dimension, (i.e. change the shape of each image from (224, 224, 3) to (1, 224, 224, 3)
    # This shape is suitable for training
    img = np.expand_dims(img, axis=0)
    # Apply preprocessing for the image, so that the training is faster
    img = xception.preprocess_input(img)
    return img

Create a  dataframe that contains a list of the file names and the corresponding category. Category 1 means lion, 0 means no lion

In [ ]:
filenames = os.listdir(imgs_path)

categories = []

for filename in filenames:
    category = filename.split('.')[0]
    if category == 'lion':
        categories.append(1)
    else:
        categories.append(0)

df = pd.DataFrame({
    'filename': filenames,
    'category': categories
})

print(df.head())

print('number of elements = ', len(df))

In [ ]:
# see sample image, you can run the same cell again to get a different image
sample = random.choice(filenames)
randomimage = image.load_img(imgs_path + sample)
plt.imshow(randomimage)

# Create the Pretrained Model
There are two approaches for transfer learning:
1. One model where the first part of it is the pretrained model and the second part is your new model. 
1. Pass all the images through the pretrained model, to get the extracted features and then use those extrated features to    train your new model. 

Here we will use the second approach (the feature extraction approach), because a matrix which contains all our images with the shape (224, 224, 3) will not fit in our RAM. So we first pass all the images through the xception model and get the extracted features, as mentioned above the model has 1000 categories, so the output will have a size of 1000 which is much smaller than (224 X 224 X 3 = 150,528) . So if we have 100 images the training matrix will have a shape of (100, 1000)

In [ ]:
# create the xception model used for the feature extraction
model_xception = xception.Xception(
    include_top=True, input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_CHANNELS),
    weights='../input/xception/xception_weights_tf_dim_ordering_tf_kernels.h5')

We will use the EarlyStopping call back to stop our training  if the accuracy is not improving for a certain number of epochs. 
If the accuracy is not improving for a certain number of epochs then it makes sense to reduce the learning rate, we implement that using the ReduceLROnPlateau call back.

In [ ]:
# Define call backs
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

early_stop = EarlyStopping(patience=7, verbose=1)

learning_rate_reduction = ReduceLROnPlateau(patience=3, verbose=1, factor=0.5)

print('call backs defined!')

# Feature Extraction
The extract_features function will take the pretrained model used for feature extraction and a batch of images, then it will pass those images through the model to obtain the extracted features. It is better to pass a maximum of a round 1000 images at a time in order not to fill our RAM. The data_set parameter is dataframe containing filenames and the corresponding categories for each file.

# Settings for Data Augmentation 

Import ImageDataGenerator

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

Create an ImageDataGenerator instance:

In [ ]:
datagen = ImageDataGenerator(
    # featurewise_center=True,
    # featurewise_std_normalization=True,
    # rotation_range=10,
    # width_shift_range=0.2,
    # height_shift_range=0.2,
    # shear_range=0.2,
    # zoom_range=0.2,
    horizontal_flip=True,
    # fill_mode='nearest',
    brightness_range=[0.8, 1.2]  # Adjust brightness between 80% and 120
)

# Modify extract_features function for data augmentation

In [ ]:
PERFORM_AUGMENTATION = True  # Set to False to disable augmentation

In [ ]:
def extract_features(model, data_set, augment=PERFORM_AUGMENTATION):
    features = []
    all_images = []

    print(f'data set has {len(data_set)} entries')
    for data_set_entry in data_set:
        file_name = data_set_entry[0]
        category = data_set_entry[1]
        path = imgs_path + file_name
        img = image.load_img(path, target_size=(IMAGE_WIDTH, IMAGE_HEIGHT))
        x = image.img_to_array(img)
        all_images.append(x)

    # Get mean and standard deviation for normalization.
    print(f'shape of an image: {all_images[0].shape}')
    all_images_combined = np.array(all_images)
    mean = np.mean(all_images_combined, axis=(0, 1, 2))
    std = np.std(all_images_combined, axis=(0, 1, 2))

    for x in all_images:
        x = (x - mean) / std
        x = np.expand_dims(x, axis=0)

        if augment:
            # Apply augmentation
            aug_iter = datagen.flow(x, batch_size=1)
            aug_img = next(aug_iter)[0]

            aug_img = xception.preprocess_input(aug_img)
            # extract features
            pred = model.predict(np.expand_dims(aug_img, axis=0))
            features.append(pred)
        else:
            # Use original image
            x = xception.preprocess_input(x)
            pred = model.predict(x)
            features.append(pred)

    return np.array(features).reshape(len(data_set), IMAGE_FEATURES_SIZE)

We split the training set into three separate sets:
1. **The training set**: used to train our model.
2. **The validation set**: used to double check that our model is not overfitting the training set, i.e. it can also generalise    to other data other than the train data
3. **The test set**: Used to estimate the accuracy of the model on new data other than the ones the model used for training

For a competetion or for some other cases, you can split the data only to training and validation sets inorder to acheive te highest possible accuracy, without the need to properly estimate how accurate the model really is.

We split the data set as follows: 2% test set, 82% train set, 16% cross_validation set
 


In [ ]:
train_set, validate_set, test_set = np.split(
    df.sample(frac=1, random_state=43), [int(.82*len(df)), int(.96*len(df))])

# Visualize the data distribution
fig, axis = plt.subplots(1, 3)
fig.tight_layout()
fig.set_size_inches(80, 20)

ax0 = sns.countplot(data=train_set, x='category', ax=axis[0])
ax1 = sns.countplot(data=validate_set, x='category', ax=axis[1])
ax2 = sns.countplot(data=test_set, x='category', ax=axis[2])

ax0.set_title('train data', fontsize=60)
ax1.set_title('cross validation data', fontsize=60)
ax2.set_title('test data', fontsize=60)

ax0.tick_params(labelsize=55)
ax1.tick_params(labelsize=55)
ax2.tick_params(labelsize=55)

print('shape of train_set:   ', np.shape(train_set))
print('shape of validate_set:', np.shape(validate_set))
print('shape of test_set:    ', np.shape(test_set))

# convert the dataframes to numpy matrices
train_set = train_set.to_numpy()
validate_set = validate_set.to_numpy()
test_set = test_set.to_numpy()

Now we perform the feature extraction. We take chunks of the train_set (1000 at a time) and pass them through the pretrained model, and save the extracted features in train_x. We then do the same with the validation_set.

In [ ]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]


# prepare parameters for model.fit
train_x = np.zeros((len(train_set), 1000))
train_y = np.zeros((len(train_set)))

validate_x = np.zeros((len(validate_set), 1000))
validate_y = np.zeros((len(validate_set)))

chunk_size = 1000

# extract features from train_set nd save it into train_x
for i, train_set_chunk in enumerate(chunks(train_set, chunk_size)):
    train_x[(i*chunk_size): (i*chunk_size + chunk_size)
            ] = extract_features(model_xception, train_set_chunk)

print('shape of train_x: ', np.shape(train_x))

# extract features from validate_set nd save it into validate_x
for i, validate_set_chunk in enumerate(chunks(validate_set, chunk_size)):
    validate_x[(i*chunk_size): (i*chunk_size + chunk_size)
               ] = extract_features(model_xception, validate_set_chunk)

print('shape of validate_x: ', np.shape(validate_x))

We take the second column from the train_set (the category column) and save it in train_y. We then convert it to a one-hot vector using the "to_categorically" method, we need the one-hot format for the model fitting later on.

In [ ]:
# prepare train_y
train_y = train_set[:, 1]
train_y = to_categorical(train_y)

# prepare validate_y
validate_y = validate_set[:, 1]
validate_y = to_categorical(validate_y)

# Build the new model and train it
Note that it is a small model, that is because it doesn't need to learn everything from scratch because we are using the extracted features. Also learning will take just around 30 seconds using the GPU.

In [ ]:
from tensorflow.keras import layers
from tensorflow import keras

APPLY_REGULARIZATION = True  # Set to False to disable regularization

transfer_model = keras.Sequential()

transfer_model.add(keras.Input(shape=(IMAGE_FEATURES_SIZE)))
transfer_model.add(layers.Flatten())

if APPLY_REGULARIZATION:
    transfer_model.add(
        layers.Dense(100, activation='relu',
                     kernel_regularizer=keras.regularizers.l2(0.01)))
    transfer_model.add(
        layers.Dense(16, activation='relu',
                     kernel_regularizer=keras.regularizers.l1(0.001)))
else:
    transfer_model.add(
        layers.Dense(100, activation='relu')
    )
    transfer_model.add(
        layers.Dense(16, activation='relu')
    )

transfer_model.add(layers.Dense(2, activation='softmax'))

transfer_model.summary()

In [ ]:
# compile and fit the model
transfer_model.compile(loss='binary_crossentropy',
                       optimizer='adam', metrics=['accuracy'])

EPOCHS = 20

history = transfer_model.fit(x=train_x, y=train_y, batch_size=16, epochs=EPOCHS,
                             callbacks=[early_stop, learning_rate_reduction],
                             validation_data=(validate_x, validate_y))

In [ ]:
transfer_model.save_weights("model.h5")

# Visualize the training process

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 10))
ax1.plot(history.history['loss'], color='b', label="Training loss")
ax1.plot(history.history['val_loss'], color='r', label="validation loss")
ax1.set_xticks(np.arange(1, EPOCHS, 1))
ax1.set_yticks(np.arange(0, 0.14, 0.02))
ax1.legend()

ax2.plot(history.history['accuracy'], color='b', label="Training accuracy")
ax2.plot(history.history['val_accuracy'],
         color='r', label="Validation accuracy")
ax2.set_xticks(np.arange(1, EPOCHS, 1))
ax2.legend()

legend = plt.legend(loc='best')
plt.tight_layout()
plt.show()

# Predict
Predict the class for the test set and evaluate the accuracy on the test set.

In [ ]:
test_x = np.zeros((len(test_set), 1000))
test_y = np.zeros((len(test_set)))

# extract features from test_set and save it into test_x
for i, test_set_chunk in enumerate(chunks(test_set, chunk_size)):
    test_x[(i*chunk_size): (i*chunk_size + chunk_size)
           ] = extract_features(model_xception, test_set_chunk)

# prepare test_y
test_y = test_set[:, 1]
test_y = to_categorical(test_y)

assert (np.shape(test_y) == (np.shape(test_set)[0], 2))

# predict and estimate the accuracy
_, accuracy = transfer_model.evaluate(test_x, test_y)
print('accuracy on test set = ',  round((accuracy * 100), 2), '% ')

# Predict Class of Random Image
The cell below will randomly select an image from the test set, predict if it is a dog or cat. You can run the cell several times to get different images.

In [ ]:
def check_string_in_filename(filepath, search_string):
    """
    Checks if a specified string is present in a filename.

    Args:
        filepath: The path to the file.
        search_string: The string to search for within the filename.

    Returns:
        True if the string is found in the filename, False otherwise.
    """
    filename = os.path.basename(filepath)
    return search_string in filename

In [ ]:
sample = random.choice(test_set)[0]
randomimage = image.load_img(imgs_path + sample)
plt.imshow(randomimage)

loaded_image = _load_image(imgs_path + sample)
extracted_feat = model_xception.predict(loaded_image)
pred = transfer_model.predict(extracted_feat)

pred = pred[0]  # convert to array
if pred[0] >= pred[1]:
    prediction_class = 'no lion'
    prediction_percentage = pred[0]
    string_to_find = "other"
    if check_string_in_filename(sample, string_to_find):
        print('CORRECT')
    else:
        print('FALSE NEGATIVE')
else:
    prediction_class = 'lion'
    prediction_percentage = pred[1]
    string_to_find = "lion"
    if check_string_in_filename(sample, string_to_find):
        print('CORRECT')
    else:
        print('FALSE POSITIVE')

print(sample)
print('I am ', int(prediction_percentage*100),
      '% sure that I am a ', prediction_class, '!', sep='')

# **Run on Real Test Images**
Specify your own directory of lion and other animal images in the realtestimgs_path to test the model on images with backgrounds not in the training, validation and test sets above. Note the test set above is simply split out from the training and validation sets so backgrounds and animals will be similar. This test allows to really test the generalization of the model

In [ ]:
realtestimgs_path = "/kaggle/input/testsnow01/testsnow01/"


def _load_image(realtestimgs_path):
    img_real = image.load_img(realtestimgs_path, target_size=(
        IMAGE_WIDTH, IMAGE_HEIGHT))  # load the image from the directory
    img_real = image.img_to_array(img_real)
    # add an additional dimension, (i.e. change the shape of each image from (224, 224, 3) to (1, 224, 224, 3)
    # This shape is suitable for training
    img_real = np.expand_dims(img_real, axis=0)
    # Apply preprocessing for the image, so that the training is faster
    img_real = xception.preprocess_input(img_real)

    return img_real

In [ ]:
filenames = os.listdir(realtestimgs_path)

categories = []

for filename in filenames:
    category = filename.split('.')[0]
    if category == 'lion':
        categories.append(1)
    else:
        categories.append(0)

df = pd.DataFrame({
    'filename': filenames,
    'category': categories
})

print(df.head())

print('number of elements = ', len(df))

from tensorflow import keras
from tensorflow.keras import layers

transfer_model = keras.Sequential()

transfer_model.add(keras.Input(shape = (IMAGE_FEATURES_SIZE)))
transfer_model.add(layers.Flatten())
transfer_model.add(layers.Dense(100, activation = 'relu', kernel_regularizer=keras.regularizers.l2(0.01)))  # L2 regularization
transfer_model.add(layers.Dense(16,  activation = 'relu', kernel_regularizer=keras.regularizers.l1(0.001))) # L1 regularization
transfer_model.add(layers.Dense(2,   activation = 'softmax'))That's it, if you find this kernel useful please upvote it :)


In [ ]:
import shutil
real_test_set = df.to_numpy()


def copy_file_to_directory(source_path, destination_directory):
    """
    Copies a file to a specified directory.

    Args:
        source_path (str): The path to the file to be copied.
        destination_directory (str): The path to the directory where the file should be copied.
    """
    try:
        # Ensure the destination directory exists; create it if not
        os.makedirs(destination_directory, exist_ok=True)

        # Copy the file to the destination directory
        shutil.copy2(source_path, destination_directory)

    except FileNotFoundError:
        print(f"Error: File '{source_path}' not found.")
    except Exception as e:
        print(f"An error occurred: {e}")

In [ ]:
failedimages_path = "/kaggle/working/failedcases/"
sortfilenames = sorted(filenames)
error_count = 0
image_count = 0
for filename in sortfilenames:
    sample = random.choice(real_test_set)[0]
    randomimage = image.load_img(realtestimgs_path + filename)
    plt.imshow(randomimage)
    plt.show(randomimage)

    loaded_image = _load_image(realtestimgs_path + filename)
    extracted_feat = model_xception.predict(loaded_image)
    pred = transfer_model.predict(extracted_feat)
    image_count = image_count + 1
    pred = pred[0]  # convert to array
    if pred[0] >= pred[1]:
        prediction_class = 'no lion'
        prediction_percentage = pred[0]
        string_to_find = "other"
        if check_string_in_filename(filename, string_to_find):
            print('CORRECT')
        else:
            print('FALSE NEGATIVE')
            copy_file_to_directory(
                realtestimgs_path + filename, failedimages_path)
            error_count = error_count + 1
    else:
        prediction_class = 'lion'
        prediction_percentage = pred[1]
        string_to_find = "lion"
        if check_string_in_filename(filename, string_to_find):
            print('CORRECT')
        else:
            print('FALSE POSITIVE')
            copy_file_to_directory(
                realtestimgs_path + filename, failedimages_path)
            error_count = error_count + 1

    print(filename)
    print('I am ', int(prediction_percentage*100),
          '% sure that I am a ', prediction_class, '!', sep='')
    print('-------------------------------------------------------------------')
print('Total Images =', image_count, 'Incorrect Images =',
      error_count, 'Accuracy =', (1-(error_count/image_count))*100)